In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.0'

# Text generation with LSTM

This notebook contains the code samples found in Chapter 8, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

[...]

## Implementing character-level LSTM text generation


Let's put these ideas in practice in a Keras implementation. The first thing we need is a lot of text data that we can use to learn a 
language model. You could use any sufficiently large text file or set of text files -- Wikipedia, the Lord of the Rings, etc. In this 
example we will use some of the writings of Nietzsche, the late-19th century German philosopher (translated to English). The language model 
we will learn will thus be specifically a model of Nietzsche's writing style and topics of choice, rather than a more generic model of the 
English language.

## Preparing the data

Let's start by downloading the corpus and converting it to lowercase:

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('말뭉치 크기:', len(text))

Corpus length: 600893


In [3]:
type(text)

str


Next, we will extract partially-overlapping sequences of length `maxlen`, one-hot encode them and pack them in a 3D Numpy array `x` of 
shape `(sequences, maxlen, unique_characters)`. Simultaneously, we prepare a array `y` containing the corresponding targets: the one-hot 
encoded characters that come right after each extracted sequence.

In [4]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('시퀀스 개수:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


## Building the network

Our network is a single `LSTM` layer followed by a `Dense` classifier and softmax over all possible characters. But let us note that 
recurrent neural networks are not the only way to do sequence data generation; 1D convnets also have proven extremely successful at it in 
recent times.

In [7]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Since our targets are one-hot encoded, we will use `categorical_crossentropy` as the loss to train the model:

In [8]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Training the language model and sampling from it


Given a trained model and a seed text snippet, we generate new text by repeatedly:

* 1) Drawing from the model a probability distribution over the next character given the text available so far
* 2) Reweighting the distribution to a certain "temperature"
* 3) Sampling the next character at random according to the reweighted distribution
* 4) Adding the new character at the end of the available text

This is the code we use to reweight the original probability distribution coming out of the model, 
and draw a character index from it (the "sampling function"):

In [9]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Finally, this is the loop where we repeatedly train and generated text. We start generating text using a range of different temperatures 
after every epoch. This allows us to see how the generated text evolves as the model starts converging, as well as the impact of 
temperature in the sampling strategy.

In [10]:
import random
import sys

# Select a text seed at random
random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60):
    print('에포크', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200278/200278 [==============================] - 80s 397us/step - loss: 2.0264
--- Generating with seed: "the slowly ascending ranks and classes, in which,
through fo"
------ temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the strenses of the most the most the stready and all the strender to the beliging that the promont to the stready the still and and the hist the menting the fact of the still the still the same the still to the stready the still the become the most the manter the gradication of the the strenses and the before the still the a the comparity in the compally to the promoned to the restint to the fa
------ temperature: 0.5
the slowly ascending ranks and classes, in which,
through for the need the conversions to should the read sumper a some and almost the contentions of what he "other in the perhaps and he mensto do in the incts as a man of higherstred the compain in the interparity and the procomposes of the instander
of 

/home/haesun/anaconda3/envs/deep-learning-with-python/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


a greatism and as a differents to a mutt the have destrue as the things but the constrained in the preceation to the make the re
------ temperature: 1.0
the slowly ascending ranks and classes, in which,
through for mankinds and .

faultive the been overdake of sprembal the one impectlat, in the eterent to re. the religions land an earth. under its law milfposting its curiestayed point, if i good in
the reaples set whe suid; of the weaker. younomination. us itself-binds and be every stcole advane of still motic incertainiture freelopmens evenswhethy can fact eperulal in difficulty,.

mining last
in himself 
------ temperature: 1.2
the slowly ascending ranks and classes, in which,
through for us. if fithinc"--arts ma of agaily its own rulspications at noh re pleamner 
a good object of
meres on tlong,intionbleard: what, may inor think of degriexce. therevilitive of will, heol-say somethings
re-lname muso tegats which yles no rerardre signacy of morally lofty--like himself
has any eccopacu

through for the same the rasine and disposin of stronger the discourse and stoon in the world to the worsh and for an opinions, and the most being interests in which he sensegoby of the doubt and in means of the dougness, and he is not one called by a great the life the man sense of the laber impires is itself about the all the present considere. the self despensible the values before with his time and
prov
------ temperature: 1.0
the slowly ascending ranks and classes, in which,
through forget in our knonced.


1menad a
stequent of seguination of
dr-all the belie sied.

tderraby
sald shoild sufficient, uswables and upon their authjecically,, and as original racichessulty express his
independany ity.


12sy duringer thereped ssireg of a neart," the means, and capable understanding koidamous
wishoususing where is to mediely at of and barings hought ideded, advanct are sellow--whan on
------ temperature: 1.2
the slowly ascending ranks and classes, in which,
through for act vage by tide w

withtu, sodes, which an, "pare that beauty suffed in times, plame not to tem of illd worch putl
epoch 16
Epoch 1/1
200278/200278 [==============================] - 79s 393us/step - loss: 1.3594
--- Generating with seed: "the slowly ascending ranks and classes, in which,
through fo"
------ temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the spirit and the spirit of the sense of the same a down a contemnate our strengthed to the same an additional string and discovered the sense and strong and social the same as the sense of the same men in the same an actions and souls and the spirits of the spirit and the soul as the conscience of the sense of the same at an advance is a more of the same the existence of the sense of the same 
------ temperature: 0.5
the slowly ascending ranks and classes, in which,
through for the world and like the foreits in the same says a sogution of the moral his problem of the present, the religion many "moral its of the came a pov

...

through formed to one is to hard of selfistes even instinct of the strift of promise the entire souls and still element in the dogmate the soul as the problem of the same things that is not to be still and consequences of the medies of process and rather the more of the soul of a man who was so that we are the philosophy of the mistaken the spirit and the away of the sensitive with the ablegation of his sel
------ temperature: 1.0
the slowly ascending ranks and classes, in which,
through fowleay and decissed of
a fowld cressifications,
leads
bror the perfication, and in their  "sympathy in cases which almost arounatic entiously, their standardings, doty doct a hardlocy that spoulld in play-a hobsery knowign against the surtify zveritioned, antige of disting
understand to their sectded to the right a backal and sense! men,
and at onother for bornervisg that the disguise long, and all
------ temperature: 1.2
the slowly ascending ranks and classes, in which,
through form in fact, for that

fronst, as noble, to regudanceals of germanly, from
henes
epoch 35
Epoch 1/1
200278/200278 [==============================] - 78s 391us/step - loss: 1.3183
--- Generating with seed: "the slowly ascending ranks and classes, in which,
through fo"
------ temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the soul and the same a good and also the soul-himself, and also the superficial the same things of the superior the same soul, and conception of the same distrust of the same the soul-that the same actional and also the sense of a distinction of the same superficial, and with the struggle and sense of the most consequently the extent the sense of the same still and sense of the same the same in
------ temperature: 0.5
the slowly ascending ranks and classes, in which,
through for the contradict of the our distort its own imagine the
realm. as it is with the same on the moral certain the soul-it disposin the majuate religion and certainty and the soul and presen

through foinean n in n no se naneieraioroan aseninoas isa  iselir ain ti arie heroroin to ar tanari a inan inon ts nina  tn oer ceaneinan ai   anie nan  norateaneia  roro sou han ar annen eronani e ttinai arer ssi o"eroneern
ina a as ano itno orananin tait ase inn teroena inaaaaninor tst orer aorat naaaa n o sr orat an an ina ananines or re inase n t "n or tesehan  inrae sreinen  an teoon n troon al tr eoi 
------ temperature: 1.2
the slowly ascending ranks and classes, in which,
through foroen oro tte   t  nr rin inanatre rrion it netr,eorar ae inoi asat in  oroiaieann onaaan reoren an nrtno ytt oine s nanan an in iaiesean airainan a it ter ae ein te aniinn rit nno init ce at enae s oanororo stniioin an in ait onoioro ien  oren inarire oronanonra s i no te  te t in aaaier  een aaar soin ttn neo anoero tion re r t enano tere aasit
raie ain tr  sti oeno sati onoan in annn  at onanie 
epoch 43
Epoch 1/1
200278/200278 [==============================] - 79s 392us/step - loss: 8.3479
--- Ge

through fopthd w o ale the thost the n ot at e ilhi  a ltr t e atethe he   inis tiof th t thit t the o-on ther t ae the  ian t hid  t the er ther  atrrtci t   the i t t thest  e  ee et t tale m e  the  thctl ote at th t  e ne trt n t he ert t thirt t tgoththe  , thherhen t i  st s[re thed twe tat t tr thert th   th t thw tth ths nt the t  th t  hete t tte tt t t thte tn th tort n  t e t oun , ae n elw theh 
------ temperature: 0.5
the slowly ascending ranks and classes, in which,
through fopthd wot b tho, io nr the he erith  thune t n ot ae  sh n r w these  o thiree
e e th ve lono-t th"thoe cf aned es mai ips  i tt sd lititre a islt hr  tilther tr her d e  rthiithe th thc s th  h the  tht r the ng  s ore taartt erted atoanre ert soirttdth t th i terentyintsi i tde r tite zoeeleert t sl t thwihie erern thtt. erj t th rht one aotk sdinois an in i  t mheniln  tnaten the enete twhn  t t
------ temperature: 1.0
the slowly ascending ranks and classes, in which,
through fopndoi"e rtd elera- r

through fo nesh saterirtothraeeuo ieni  c e  at ittoan' atss   orteaooueandaen ta  o oeeth acrels atensto "areis aineneon e   ece el drd n thhaeal asooditohe eal ditie th sstr thintt itia anol thlottalatssn e hi ttq s  andlea oohoseu e t ehhrrn inahlhr  orrnotouluelatnhnae dah n teoltaso lderainle tsat  tsls lol en t do sithyt oe s tntse raldte lrrtsesrsniethln aunraroe hle at  sistna oeed erhths r ahiaihes
------ temperature: 1.2
the slowly ascending ranks and classes, in which,
through fola oo oen ha oilhlrdro isseseo  nd uo e aatnisroihsinela aooe lanm tth aiod  hl tlle olenotanite iaista  s inuaaitaosinastue hedahonos s srntshee theo  hr  ile essdiinonoaeo eealeatie tntnero ohrir  cealondsi tdt laddt edaula io-suderaris ninle sn t ho itt o th ieisiaaoahhor totaln siasteor hl ale  raniihdith nt inatfe  oaonldooihes ut d o  oaas o ouaao sud hhre arair as tns ren e hoshelsier sa ns
epoch 58
Epoch 1/1
200278/200278 [==============================] - 78s 392us/step - loss: 5.1521
--- Ge


As you can see, a low temperature results in extremely repetitive and predictable text, but where local structure is highly realistic: in 
particular, all words (a word being a local pattern of characters) are real English words. With higher temperatures, the generated text 
becomes more interesting, surprising, even creative; it may sometimes invent completely new words that sound somewhat plausible (such as 
"eterned" or "troveration"). With a high temperature, the local structure starts breaking down and most words look like semi-random strings 
of characters. Without a doubt, here 0.5 is the most interesting temperature for text generation in this specific setup. Always experiment 
with multiple sampling strategies! A clever balance between learned structure and randomness is what makes generation interesting.

Note that by training a bigger model, longer, on more data, you can achieve generated samples that will look much more coherent and 
realistic than ours. But of course, don't expect to ever generate any meaningful text, other than by random chance: all we are doing is 
sampling data from a statistical model of which characters come after which characters. Language is a communication channel, and there is 
a distinction between what communications are about, and the statistical structure of the messages in which communications are encoded. To 
evidence this distinction, here is a thought experiment: what if human language did a better job at compressing communications, much like 
our computers do with most of our digital communications? Then language would be no less meaningful, yet it would lack any intrinsic 
statistical structure, thus making it impossible to learn a language model like we just did.


## Take aways

* We can generate discrete sequence data by training a model to predict the next tokens(s) given previous tokens.
* In the case of text, such a model is called a "language model" and could be based on either words or characters.
* Sampling the next token requires balance between adhering to what the model judges likely, and introducing randomness.
* One way to handle this is the notion of _softmax temperature_. Always experiment with different temperatures to find the "right" one.